In [ ]:
import json
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# Define the file path
directory = '../../../ARC-AGI/data/training/'


# Convert 'train' and 'test' data within the task dictionary from lists to numpy arrays
def convert_task_to_numpy_arrays(task):
    if 'train' in task:
        for i in range(len(task['train'])):
            task['train'][i]['input'] = np.array(task['train'][i]['input'], dtype=np.uint8)
            task['train'][i]['output'] = np.array(task['train'][i]['output'], dtype=np.uint8)
    if 'test' in task:
        for i in range(len(task['test'])):
            task['test'][i]['input'] = np.array(task['test'][i]['input'], dtype=np.uint8)
            task['test'][i]['output'] = np.array(task['test'][i]['output'], dtype=np.uint8)
    
    return task

# Load the JSON file
def safe_load_json(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError as e:
        print(f"File not found: {file_path}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {file_path}: {e}")
        return None
    
# Load Files concurrently
def load_all_json_files_concurrently(directory):
    json_files_names = list(entry.name for entry in os.scandir(directory) if entry.is_file() and entry.name.endswith('.json'))
    json_files_paths = [os.path.join(directory, file) for file in json_files_names]
    
    with ThreadPoolExecutor() as executor:
        data = dict(zip(json_files_names, executor.map(safe_load_json, json_files_paths)))
    
    return data

# Print the loaded data
data = load_all_json_files_concurrently(directory)


In [ ]:
# Example of how to access the data
task = data["f35d900a.json"]
print(task)


formatted_task = convert_task_to_numpy_arrays(task)
print (formatted_task)

In [ ]:
test_data_point = data["29ec7d0e.json"]['test'][0]["input"]

In [ ]:
import cv2
import matplotlib.pyplot as plt

def convert_to_image(data_point):
    # Define a color map for numbers 0 to 9
    color_map = {
        0: (0, 0, 0),       # Black
        1: (255, 146, 30),  # Blue
        2: (49, 60, 248),   # Red
        3: (46, 205, 79),   # Green
        4: (0, 220, 254),   # Yellow
        5: (153, 153, 153), # Gray
        6: (164, 58, 229),  # Pink 
        7: (26, 132, 255),  # Orange
        8: (242, 216, 135), # Light Blue
        9: (51, 17, 149),   # Maroon
    }
    # Convert the data point to a NumPy array if it's not already
    if not isinstance(data_point, np.ndarray):
        data_point = np.array(data_point, dtype=np.uint8)

    # Create a color map array
    color_map_array = np.array([color_map[i] for i in range(10)], dtype=np.uint8)

    # Map the numbers to colors using advanced indexing
    image = color_map_array[data_point]
    return image

# Helper display function for notebooks
def display_image(image):
    # Convert the image from BGR (OpenCV format) to RGB (matplotlib format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image using matplotlib
    plt.imshow(image)
    plt.axis('off')  # Hide the axis
    plt.show()
   

# Example usage
image = convert_to_image(test_data_point)
display_image(image)